# Importation de la library necessaire

In [2]:
import numpy as np
import pandas as pd 
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
#import cv2 
import glob
import random
from os import listdir
from sklearn.metrics import classification_report
import tensorflow as tf
import keras.utils as image

# Liste des dossiers de patients/ nombre de patient

In [18]:
pat_fold = "F:/Data_Breast_Cancer/IDC_regular_ps50_idx5/"
folder = listdir(pat_fold)
len(folder)

279

# Comptons le notre total d'images que nous avons

In [24]:
# NOtre total d'image
img_T = 0
for n in range(len(folder)):
    patient_id = folder[n]
    for c in [0, 1]:
        patient_path = pat_fold + patient_id 
        class_path = patient_path + "/" + str(c) + "/"
        subfiles = listdir(class_path)
        img_T += len(subfiles)
img_T

277524

In [27]:
# Extraction de toutes les images
#img_cancer = glob.glob('F:/Data_Breast_Cancer/IDC_regular_ps50_idx5/**/*.png', recursive = True)

#for imgname in img_cancer[:10]:
    #print(imgname)

Notre base de données comporte 277524 images collecté chez 279 patients. Nous allons regroupé ces images en fonction du Id des patients, le lien d'accès a chaque image ainsi que le résultat d'examen de chacune en Dataframe pour faciliter leur manipulation. 

In [30]:
# Créer un DataFrame vide
df = pd.DataFrame(index=np.arange(0, img_T), columns=["patient_id", "path", "target"])

i = 0
for patient_id in folder:
    patient_path = pat_fold + patient_id 
    for target in [0, 1]:
        class_path = patient_path + "/" + str(target) + "/"
        subfiles = listdir(class_path)
        for image_path in subfiles:
            # Chemin complet de l'image
            image_fullpath = class_path + image_path
            df.loc[i, "path"] = image_fullpath
            df.loc[i, "target"] = target
            df.loc[i, "patient_id"] = patient_id
            i += 1             
df.head()


,patient_id,path,target
0,10253,F:/Data_Breast_Cancer/IDC_regular_ps50_idx5/10...,0
1,10253,F:/Data_Breast_Cancer/IDC_regular_ps50_idx5/10...,0
2,10253,F:/Data_Breast_Cancer/IDC_regular_ps50_idx5/10...,0
3,10253,F:/Data_Breast_Cancer/IDC_regular_ps50_idx5/10...,0
4,10253,F:/Data_Breast_Cancer/IDC_regular_ps50_idx5/10...,0


Pour chaque patient donné, nous pouvons acceder à chaque image  et avoir le résultat de chaque image

In [35]:
df['target'].value_counts()

target
0    198738
1     78786
Name: count, dtype: int64

# Exploration et analyse de données

In [ ]:
cancer_img = glob.glob('F:/Data_Breast_Cancer/IDC_regular_ps50_idx55/**/*.png', recursive = True)

for imgname in cancer_img[:3]:
    print(imgname)

In [36]:
N_IDC = []
P_IDC = []

for img in df:
    if img[-5] == '0' :
        N_IDC.append(img)
    
    elif img[-5] == '1' :
        P_IDC.append(img)
plt.figure(figsize = (15, 15))

some_non = np.random.randint(0, len(N_IDC), 18)
some_can = np.random.randint(0, len(P_IDC), 18)

s = 0
for num in some_non:
    
        img = image.load_img((N_IDC[num]), target_size=(100, 100))
        img = image.img_to_array(img)
        
        plt.subplot(6, 6, 2*s+1)
        plt.axis('off')
        plt.title('no cancer')
        plt.imshow(img.astype('uint8'))
        s += 1
s = 1
for num in some_can:
    
        img = image.load_img((P_IDC[num]), target_size=(100, 100))
        img = image.img_to_array(img)
        
        plt.subplot(6, 6, 2*s)
        plt.axis('off')        
        plt.title('IDC (+)')
        plt.imshow(img.astype('uint8'))
        s += 1


IndexError: string index out of range

In [ ]:


# Création du modèle séquentiel
model = Sequential()

# Première couche de convolution suivie de MaxPooling
model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(25, 25, 3)))
model.add(MaxPooling2D((2, 2)))

# Deuxième couche de convolution suivie de MaxPooling
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2)))

# Troisième couche de convolution suivie de MaxPooling
model.add(Conv2D(128, (3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2)))

# Aplatissement des données pour les couches fully connected
model.add(Flatten())

# Première couche fully connected avec dropout
model.add(Dense(256, activation='relu'))
model.add(Dropout(0.5))

# Deuxième couche fully connected pour la classification
#model.add(Dense(1, activation='sigmoid'))
model.add(Dense(2, activation='sigmoid'))

# Compilation du modèle
#model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])


# Compilation du modèle avec l'optimiseur Adam et un taux d'apprentissage spécifié
model.compile(optimizer=Adam(learning_rate=0.001), loss='binary_crossentropy', metrics=['accuracy'])


# Affichage de la structure du modèle
model.summary()


In [ ]:
from skimage.transform import resize
# Redimensionner les images à la taille attendue (25x25)
X_resized = np.array([resize(image, (25, 25)) for image in X])

# Assurez-vous que les valeurs des pixels sont comprises entre 0 et 1
X_resized = X_resized.astype('float32') / 255.0

In [ ]:


# Définir le nombre de folds
k = 2

# Créer un objet StratifiedKFold
kf = StratifiedKFold (n_splits=k, shuffle=True)

# Liste pour stocker les scores de validation pour chaque fold
val_scores = []

# Boucle sur les folds
for fold, (train_index, test_index) in enumerate(kf.split(X, y), 1):
    # Diviser les données en ensembles d'entraînement et de validation pour ce fold
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]
    
    # Créer, compiler et entraîner votre modèle
    model = Sequential()
    model.add(Conv2D(filters=32, kernel_size=(4, 4), activation='relu', input_shape=(25, 25, 3)))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Flatten())
    model.add(Dense(256, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(1, activation='sigmoid'))  # Couche de sortie pour la classification binaire
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

   
    # Entraînement du modèle avec validation croisée
    history = model.fit(X_train, y_train, epochs=5, batch_size=32, validation_data=(X_test, y_test))
    
    # Afficher les courbes de perte et de précision pour l'entraînement et la validation
    plt.figure(figsize=(10, 4))
    plt.subplot(1, 2, 1)
    plt.plot(history.history['loss'], label='Training Loss')
    plt.plot(history.history['val_loss'], label='Validation Loss')
    plt.title('Fold {} - Loss'.format(fold))
    plt.xlabel('Epochs')
    plt.ylabel('Loss')
    plt.legend()
    
    plt.subplot(1, 2, 2)
    plt.plot(history.history['accuracy'], label='Training Accuracy')
    plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
    plt.title('Fold {} - Accuracy'.format(fold))
    plt.xlabel('Epochs')
    plt.ylabel('Accuracy')
    plt.legend()
    
    plt.tight_layout()
    plt.show()

    # Évaluer le modèle sur l'ensemble de validation et enregistrer le score
    _, val_score = model.evaluate(X_test, y_test)
    val_scores.append(val_score)

    # Calculer la moyenne des scores de validation
mean_val_score = np.mean(val_scores)
print("Mean validation score:", mean_val_score)

